# Writing a Device driver
### Basic structure
Here is a simple (but complete and functional) code block that implements a VISA driver for a power sensor:

In [1]:
import labbench as lb
import pandas as pd

# Specific driver definitions are implemented by subclassing classes like lb.VISADevice
class PowerSensor(lb.VISADevice):
    initiate_continuous = lb.property.bool(key='INIT:CONT')
    output_trigger      = lb.property.bool(key='OUTP:TRIG')
    trigger_source      = lb.property.str(key='TRIG:SOUR', only=['IMM','INT','EXT','BUS','INT1'])
    trigger_count       = lb.property.int(key='TRIG:COUN', min=1,max=200,step=1)
    measurement_rate    = lb.property.str(key='SENS:MRAT', only=['NORM','DOUB','FAST'])
    sweep_aperture      = lb.property.float(key='SWE:APER',  min=20e-6, max=200e-3,help='time (in s)')
    frequency           = lb.property.float(key='SENS:FREQ', min=10e6, max=18e9,help='center frequency (Hz)')

    def preset (self):
        """ Apply the instrument's preset state.
        """
        self.write('SYST:PRES')

    def fetch (self):
        """ Get already-acquired data from the instrument.

        Returns:
            The data trace packaged as a pd.DataFrame
        """
        response = self.query('FETC?').split(',')
        if len(response)==1:
            return float(response[0])
        else:
            return pd.to_numeric(pd.Series(response))

Let's work through what this does.

### 1. Every `labbench` driver is a subclass of a labbench Device class, such as lb.VISADevice:

This is the definition of the PowerSensor:
```python
class PowerSensor(lb.VISADevice):
    # ...
```
This single line gives our power sensor driver all of the general capabilities of a VISA driver this driver class (known as "subclassing" "inheriting" in software engineering). This means that in this one line, the PowerSensor driver has adopted _all of the same member and attribute features as a "plain" VISADevice_. The `VISADevice` class helps streamline use of the `pyvisa` with features like
* managing connection and disconnection, given a VISA resource string;
* shortcuts for accessing simple instrument states, implemented entirely based on definitions (discussed below); and
* wrapper methods (i.e., member functions) for pyvisa resource `write` and `query` methods.

A more complete listing of everything that comes with `lb.VISADevice` is in the [programming reference](http://ssm.ipages.nist.gov/labbench/labbench.html#labbench.backends.VISADevice).

This power sensor driver definition is just that - a definition. To _use_ the driver and connect to the instrument in the lab, instantiate it and connect to the device. This is the simplest recommended way to instantiate, connect, and then disconnect in a script:
```python
# Here is the `with` block
with PowerSensor('TCPIP::10.0.0.1::::INSTR') as sensor:
    pass
    # The sensor is connected in this "with" block. Afterward, it disconnects, even
    # if there is an exception. Automation code that uses the sensor would go here.

# Now the `with` block is done and we're disconnected
print('Disconnected, all done')
```
It's nice to leave the sensor connected sometimes, like for interactive play on a python prompt. In that case, you can manually connect and disconnect:
```python
sensor = PowerSensor('TCPIP::10.0.0.1::::INSTR')
sensor.connect()
# The sensor is connected now. Automation code that uses the sensor would go here.
sensor.disconnect() # We have to manually disconnect when we don't use a with block.
print('Disconnected, all done')
```

There are two key pieces here:
* The instantiation, `PowerSensor('TCPIP::10.0.0.1::::INSTR')`, is where we create a power sensor object that we can interact with. All `VISADevice` drivers use this standard resource string formatting; other types of drivers have different formats.
* The `with` block (talked about under the name _context management_ in python language documents) serves two functions for any labbench driver (not just VISADevice):
    1. The instrument is connected at the start of the with block
    2. guarantees that the instrument will be disconnected after the with end of the with block, _even if there is an error inside the block!_
    

    
### 2. Getting and setting simple parameters in the device the `state` object
##### Reading the definition
Each driver has an attribute called `state`. It is an optional way to give your users shortcuts to get and set simple instrument settings. This is the definition from the example above:

```python
    initiate_continuous = lb.Bool      (key='INIT:CONT')
    output_trigger      = lb.Bool      (key='OUTP:TRIG')
    trigger_source      = lb.EnumBytes (key='TRIG:SOUR', values=['IMM','INT','EXT','BUS','INT1'])
    trigger_count       = lb.Int       (key='TRIG:COUN', min=1,max=200,step=1)
    measurement_rate    = lb.EnumBytes (key='SENS:MRAT', values=['NORM','DOUB','FAST'])
    sweep_aperture      = lb.Float     (key='SWE:APER',  min=20e-6, max=200e-3,help='time (in s)')
    frequency           = lb.Float     (key='SENS:FREQ', min=10e6, max=18e9,help='input center frequency (in Hz)')
```

The `VISADevice` driver uses the metadata given for each descriptor above to determine how to communicate with the remote instrument on assignment. Behind the scenes, the `state` object has extra features that can monitor changes to these states to automatically record the changes we make to these states to a database, or (in the future) automatically generate a GUI front-panel.

*Every* labbench driver has a state object, including at least the boolean state called `connected` (indicating whether the host computer is connected with the remote device is connected or not).

---
##### Using state attributes

Making an instance of PowerSensor - in the example, this was `PowerSensor('TCPIP::10.0.0.1::::INSTR')` - causes the `state` object to become interactive.

Assignment causes causes the setting to be applied to the instrument. For example, 
`sensor.state.initiate_continuous = True` makes machinery inside `lb.VISADevice` do the following:
1. validate that `True` is a valid python boolean value (because we defined it as `lb.Bool`)
2. convert the python boolean `True` to a string (because `lb.VISADevice` knows SCPI uses string commands)
3. send the SCPI string `'INIT:CONT TRUE'` (because we told it the command string is `'INIT:CONT'`, and by default it assumes that settings should be applied as `'<command> <value>'`)

Likewise, a parameter "get" operation is triggered by simply using the attribute. The statement `print(sensor.state.initiate_continuous)` triggers `lb.VISADevice` to do the following:
1. an SCPI query with the string `'INIT:CONT?'` (because we told it the command string is `'INIT:CONT'`, and by default it assumes that settings should be applied as `'<command>?'` with return values reported in a response string),
2. the response string is converted to a python boolean type (because we defined it as `lb.Bool`),
3. the converted boolean value is passed to the `print` function for display.

##### Example of assigning to and from states
Here is working example that gets and sets parameter values by communicating with the device.

```python
with PowerSensor('TCPIP::10.0.0.1::::INSTR') as sensor:

    # This prints True if we're still in the with block
    print(sensor.state.isopen)    
    
    # Use SCPI to request the identity of the sensor,
    # then return and print it. This was inherited from
    # VISADevice, so it is available on any VISADevice driver.
    print(sensor.state.identity)
    
    # PowerSensor.state.frequency is defined as a float. Assigning
    # to it causes logic inherited from lb.VISADevice
    # to convert this to a string, and then write the SCPI string
    # 'SENS:FREQ 2.45e9' to the instrument.
    sensor.state.frequency = 2.45e9 # Set the power sensor center frequency to 2.45e9 GHz
    
    # We can also access the remote value of sensor.state.frequency.
    # Behind the scenes, each time we fetch the value, magic in
    # lb.VISADevice retrieves the current value from the instrument
    # with the SCPI query 'SENS:FREQ?', and then converts it to a floating point
    # number.
    print('The sensor frequency is {} GHz'.format(sensor.state.frequency/1e9))
    
print(sensor.state.isopen) # Prints False - we're disconnected
```
Simply put: assigning to or from with the attribute in the driver state instance causes remote set or get operations. The python data type matches the definition in the `state` class.

##### Discovering and navigating states
Inheriting from `VISADevice` means that `PowerSensor.state` includes the seven states defined here, plus all others listed provided by VISADevice.state. Since these aren't listed here, it can get confusing tracking what has been inherited (like in other object-oriented libraries). Fortunately, there are many ways to explore the entire list of states that have been inherited from the parent state class:
1. Look it up [in the API reference manual](http://ssm.ipages.nist.gov/labbench/labbench.html#labbench.visa.VISADevice.state)
2. When working with an instantiated driver object in an ipython or jupyter notebook command prompt, type `lb.VISADevice.state.` and press tab to autocomplete a list of valid options. You'll also see some functions to do esoteric things with these states.
3. When working in an editor like pycharm or spyder, you can ctrl+click on the right side of `VISADevice.state` to skip directly to looking at the definition of `VISADevice.state` in the `labbench` library
4. When working in any kind of python prompt, you can use the `help` function
   ```python
   help(PowerSensor.state)
   ```
5. When working in an ipython or jupyter prompt, a nicer option than 4. is the ipython help magick:
   ```python
   PowerSensor.state?
   ```

##### Writing state attributes
The way we code this is a little unusual outside of python packages for web development. When we write a driver class, we add attributes defined with helper information such as
- the python type that should represent the parameter
- bounds for acceptable values of the parameter
- descriptive "help" information for the user

These attributes are a kind of python type state class is a _descriptor_. We call them "traits" because following an underlying library that we extend, [traitlets](https://github.com/ipython/traitlets) under the hood. The example includes seven state traits.

After instantiating with `PowerSensor()`, we can start interacting with `sensor.state`. Each one is now a live object we can assign to and use like any other python object. The difference is, each time we get the value, it is queried from the instrument, and each time we assign to it (the normal `=` operator), a set command goes to the instrument to set it.

The definition above includes metadata that dictates the python data type handled for this assignment operation, and how it should be converted:

| **Descriptor metadata type**    	| **Uses in `PowerSensor` example**  | **Behavior depends on the Device implementation** 	|
|---------------------------------	|------------------------------------|-----------------------------------	|
| Python data type for assignment 	| `lb.Float`, `lb.EnumBytes`, etc.	 | No                          	        |
| Data validation settings        	| `min`,`max`,`step` (for numbers)   | No                                   |
|                                   | `values` (for enumerated types) 	 | No                                	|
| Documentation strings           	| `help`                             | No                                	|
| Associated backend command      	| `command`                          | Yes                               	|

Some types of drivers ignore `command` keyword, as discussed in [how to write a labbench device driver](how to write a device driver).



### 3. Device methods for commands and data acquisition
The `state` class above is useful for remote assignment operations on simple scalar data types. Supporting a broader collection of operation types ("trigger a measurement," "fetch and return measurement data," etc.) need the flexibility of more general-purpose functions. In python, a member function of a class is called a method.

Here are the methods defined in `PowerSensor`:
```python
def preset (self):
    self.write('SYST:PRES')

def fetch (self):
    response = self.query('FETC?').split(',')
    if len(response)==1:
        return float(response[0])
    else:
        return pd.to_numeric(pd.Series(response))
```
These are the methods that are specific to our power sensor device. 
* The `preset` function tells the device to revert to its default state.
* The `fetch` method performs some text processing on the response from the device, and returns either a single scalar or a pandas Series if the result is a sequence of power values.

The `labbench` convention is that the names of these methods are verbs (or sentence predicates, when single words are not specific enough).

##### Example data acquisition script
Here is an example that presets the device, sets the center frequency to 2.45 GHz, and then collects 10 power samples:

In [ ]:
with PowerSensor('TCPIP::10.0.0.1::::INSTR') as sensor:
    print('Connected to power sensor {}'.format(sensor.state.identity))
    
    sensor.preset()
    sensor.wait()   # VISADevice includes in the standard VISA wait method, which sends SCPI '*WAI'
    sensor.state.frequency = 2.45e9 # Set the power sensor center frequency to 2.45e9 GHz
    
    power_levels = pd.Series([sensor.fetch() for i in range(10)])

print('All done! Got these power levels: ')
print(power_levels) 

##### Discovering and navigating device driver methods
Inheritance has similar implications as it does for the `VISADevice.state` class. Inheriting from `VISADevice` means that `PowerSensor` includes the `preset` and `fetch` methods, plus many more from `lb.VISADevice` (some of which it inherited from `lb.Device`). Since these aren't listed in the example definition above, it can get confusing tracking what methods are available through inheritance (like in other object-oriented libraries). Sometimes, informally, this confusion is called "abstraction halitosis." Fortunately, there are many ways to identify the available objects and methods:
1. Look it up [in the API reference manual](http://ssm.ipages.nist.gov/labbench/labbench.html#labbench.visa.VISADevice)
2. When working with an instantiated driver object in an ipython or jupyter notebook command prompt, type `lb.VISADevice.` and press tab to autocomplete a list of valid options. You'll also see some functions to do esoteric things with these states.
3. When working in an editor like pycharm or spyder, you can ctrl+click on the right side of `VISADevice` to skip directly to looking at the definition of `VISADevice` in the `labbench` library
4. When working in any kind of python prompt, you can use the `help` function
   ```python
   help(PowerSensor)
   ```
5. When working in an ipython or jupyter prompt, a nicely formatted version of 4. is the ipython help magick:
   ```python
   PowerSensor?
   ```

## Miscellaneous extras
##### Connecting to multiple devices
The best way to connect to multiple devices is to use a single `with` block. For example, a 10-sample acquisition with two power sensors might look like this:

In [ ]:
with PowerSensor('TCPIP::10.0.0.1::::INSTR') as sensor1,\
     PowerSensor('TCPIP::10.0.0.2::::INSTR') as sensor2:
    print('Connected to power sensors')
    
    for sensor in sensor1, sensor2:
        sensor.preset()
        sensor.wait()   # VISADevice includes in the standard VISA wait method, which sends SCPI '*WAI'
        sensor.state.frequency = 2.45e9 # Set the power sensor center frequency to 2.45e9 GHz
    
    power_levels = pd.DataFrame([[sensor1.fetch(),sensor2.fetch()] for i in range(10)])

print('All done! Got these power levels: ')
print(power_levels) 

##### Execute a function on state changes
Database management and user interface tools make extensive use of callbacks, which gives an opportunity for you to execute custom code any time an assignment causes a state to change. A state change can occur in a couple of ways: 
* This triggers a callback if 2.45e9 is different than the last observed frequency:
  ```python
     sensor.state.frequency = 2.45e9
  ```
* This triggers a callback if the instrument returns a frequency that is is different than the last observed frequency
  ```python
     current_freq = sensor.state.frequency 
  ```
  
Configure a function call on an observed change with the `observe` method in `sensor.state`:

In [ ]:
def callback(change):
    """ the callback function is given a single argument. change
        is a dictionary containing the descriptor ('frequency'),
        the state instance that contains frequency, and both
        the old and new values.
    """
    # insert GUI update here?
    # commit updated state to a database here?
    print(change)

with PowerSensor('TCPIP::10.0.0.1::::INSTR') as sensor:
    sensor.state.observe(callback)
    
    sensor.preset()
    sensor.wait()   # VISADevice includes in the standard VISA wait method, which sends SCPI '*WAI'
    sensor.state.frequency = 2.45e9 # Set the power sensor center frequency to 2.45e9 GHz    

print('All done! Got these power levels: ')
print(power_levels)

Use of callbacks can help separate the actual measurement loop (the contents of the `with` block) from other functions for debugging, GUI, and database management. The result can be code that is more clear.